## 问题定义

问题定义如下图，矩形求解区域，长宽为0.1m，上侧面为从左至右的1m/s的滑移边界，其他三个侧面为无滑移边界。雷诺数为10。
![Problem](./resource/problem_define.png)

对待求解问题的数学描述如下：
![Equation](./resource/equation.png)

其中，$u,v$分别为x方向速度，y方向速度，$p$为压强，$\nu,\rho$分别为运动粘度和密度。在本求解中定义$\nu=0.01$，$\rho=1.0$

### 求解目标

给定坐标$(x,y)$求解稳态结果（$u,v, p$）

## Domain Decomposition

本例的目的是引入域分解方法

整个求解域被分解为两部分，两部分单独求解


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [2]:
# 必要的符号运算
from sympy import Symbol, Eq, Abs, Heaviside

import modulus.sym

# 超参数
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.hydra.utils import compose

# 求解器
from modulus.sym.solver import Solver

# domain
from modulus.sym.domain import Domain

# 几何物体
from modulus.sym.geometry.primitives_2d import Rectangle

# 约束
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)

# validator
from modulus.sym.domain.validator import PointwiseValidator

# inferencer
from modulus.sym.domain.inferencer import PointwiseInferencer
from modulus.sym.key import Key

# Equation
from modulus.sym.eq.pdes.navier_stokes import NavierStokes

# post process
from modulus.sym.utils.io import (
    csv_to_dict,
    ValidatorPlotter,
    InferencerPlotter,
)
import matplotlib.pyplot as plt

from modulus.sym.node import Node
import copy

In [3]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 10000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: 1000
  rec_inference_freq: 2000
  rec_monitor_freq: 1000
  rec_constraint_freq: 2000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: true
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch:
  fully_connected:
    arch_type: fully_connected
    input_keys: ???
  

### 定义必要组件

#### PDE

In [4]:
ns = NavierStokes(nu=0.01, rho=1.0, dim=2, time=False)

# 每一个域均需要有独立的NS方程
def generate_pde_copies(eq, num_copies=2):  # deep copy所有方程以及变量
    """
    Generate multiple copies of a equation to use it
    for different domains.
    """

    from sympy import Function

    eq_copies = []
    for i in range(num_copies):
        temp_eq = copy.deepcopy(eq)

        # Find all the functions that need substituion
        eq_sum = 0
        for k, v in temp_eq.equations.items():
            eq_sum += v  # Generate a single equation to find all the terms.

        for func in eq_sum.atoms(Function):
            func_new_str = func.func.__name__ + "_" + str(i + 1)
            args = func.args
            func_new = Function(func_new_str)(*args)
            temp_eq.subs(func, func_new)

        # Generate the functions to be substituted
        equations_new = {}
        for k, v in temp_eq.equations.items():
            equations_new[k + "_" + str(i + 1)] = v

        # Replace the equations dict
        temp_eq.equations = equations_new
        eq_copies.append(temp_eq)

    return eq_copies

copies = generate_pde_copies(ns, num_copies=2)
for eq in copies:
    print(eq.equations)

{'continuity_1': 1.0*Derivative(u_1(x, y), x) + 1.0*Derivative(v_1(x, y), y), 'momentum_x_1': 1.0*u_1(x, y)*Derivative(u_1(x, y), x) + 1.0*v_1(x, y)*Derivative(u_1(x, y), y) + Derivative(p_1(x, y), x) - 0.01*Derivative(u_1(x, y), (x, 2)) - 0.01*Derivative(u_1(x, y), (y, 2)), 'momentum_y_1': 1.0*u_1(x, y)*Derivative(v_1(x, y), x) + 1.0*v_1(x, y)*Derivative(v_1(x, y), y) + Derivative(p_1(x, y), y) - 0.01*Derivative(v_1(x, y), (x, 2)) - 0.01*Derivative(v_1(x, y), (y, 2))}
{'continuity_2': 1.0*Derivative(u_2(x, y), x) + 1.0*Derivative(v_2(x, y), y), 'momentum_x_2': 1.0*u_2(x, y)*Derivative(u_2(x, y), x) + 1.0*v_2(x, y)*Derivative(u_2(x, y), y) + Derivative(p_2(x, y), x) - 0.01*Derivative(u_2(x, y), (x, 2)) - 0.01*Derivative(u_2(x, y), (y, 2)), 'momentum_y_2': 1.0*u_2(x, y)*Derivative(v_2(x, y), x) + 1.0*v_2(x, y)*Derivative(v_2(x, y), y) + Derivative(p_2(x, y), y) - 0.01*Derivative(v_2(x, y), (x, 2)) - 0.01*Derivative(v_2(x, y), (y, 2))}


#### Model

In [5]:
# 定义简单的全连接网络
# 输入为坐标x和y
# 输出为两个方向的速度u,v以及压强p

# 两个域需要两个模型
flow_net_1 = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u_1"), Key("v_1"), Key("p_1")],
    cfg=cfg.arch.fully_connected,
)
flow_net_2 = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u_2"), Key("v_2"), Key("p_2")],
    cfg=cfg.arch.fully_connected,
)

print(flow_net_1)

FullyConnectedArch(
  (_impl): FullyConnectedArchCore(
    (layers): ModuleList(
      (0): FCLayer(
        (linear): WeightNormLinear(in_features=2, out_features=512, bias=True)
      )
      (1-5): 5 x FCLayer(
        (linear): WeightNormLinear(in_features=512, out_features=512, bias=True)
      )
    )
    (final_layer): FCLayer(
      (activation_fn): Identity()
      (linear): Linear(in_features=512, out_features=3, bias=True)
    )
  )
)


#### Interface Node

In [6]:
# 两个区域的界面损失
interface_nodes = [Node.from_sympy(Symbol("u_1") - Symbol("u_2"), "dirichlet_u")]
interface_nodes += [Node.from_sympy(Symbol("v_1") - Symbol("v_2"), "dirichlet_v")]
interface_nodes += [Node.from_sympy(Symbol("p_1") - Symbol("p_2"), "dirichlet_p")]

#### 推理节点

最终的解是两个独立模型的组合

In [7]:
# Heaviside： 阶跃函数
custom_nodes = [
    Node.from_sympy(
        Symbol("u_1") * Heaviside(-Symbol("y"))
        + Symbol("u_2") * Heaviside(Symbol("y")),
        "u",
    )
]
custom_nodes += [
    Node.from_sympy(
        Symbol("v_1") * Heaviside(-Symbol("y"))
        + Symbol("v_2") * Heaviside(Symbol("y")),
        "v",
    )
]
custom_nodes += [
    Node.from_sympy(
        Symbol("p_1") * Heaviside(-Symbol("y"))
        + Symbol("p_2") * Heaviside(Symbol("y")),
        "p",
    )
]

#### 整合节点


In [8]:
nodes = (
    copies[0].make_nodes()  # 方程0的计算图
    + copies[1].make_nodes()  # 方程1的计算图
    + interface_nodes  # 界面损失节点
    + [flow_net_1.make_node(name="flow_network_1")]  # 模型节点1
    + [flow_net_2.make_node(name="flow_network_2")]  # 模型节点2
)

In [9]:
# 推理节点图
nodes_infer = custom_nodes

#### Geo

In [10]:
# 长宽均为1的矩形
height = 0.1
width = 0.1
x, y = Symbol("x"), Symbol("y")

# 定义两个子区域
rec_1 = Rectangle((-width / 2, -height / 2), (width / 2, 0))
rec_2 = Rectangle((-width / 2, 0), (width / 2, height / 2))

# 整个区域
rec = Rectangle((-width / 2, -height / 2), (width / 2, height / 2))

#### Domain

在Domain中定义约束以及训练所需的各种组件

In [11]:
# make ldc domain
ldc_domain = Domain()

#### 边界条件

四个边界条件

In [12]:
# 顶部滑移边界
# 水平速度为1.0，垂直速度为0
# 注意lambda_weighting参数，这个参数定义了样本权重，在本示例中，约靠近边界，样本的权重越小
# criteria定义了采样的位置
# outvar定义了边界条件

# 上边界全部在模型2的范围内
top_wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec_2,
    outvar={"u_2": 1.0, "v_2": 0},
    batch_size=cfg.batch_size.TopWall,
    lambda_weighting={
        "u_2": 1.0 - 20 * Abs(x),
        "v_2": 1.0,
    },  # weight edges to be zero
    criteria=Eq(y, height / 2),
)
ldc_domain.add_constraint(top_wall, "top_wall")


# 左右以及下侧的无滑移边界
# 这里简化了写法
# 实际上两个模型在三个边界上均有监督
no_slip = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u_1": 0, "v_1": 0, "u_2": 0, "v_2": 0},
    batch_size=cfg.batch_size.NoSlip,
    criteria=y < height / 2,
)
ldc_domain.add_constraint(no_slip, "no_slip")

#### PDE约束

内部满足PDE约束

In [13]:
# interior
# 由于本求解中NS方程直接调用的模型，所以这里outvar直接使用了对应的key
# 具体定义可参考modulus/eq/pdes/navier_stokes.py
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"continuity_2": 0, "momentum_x_2": 0, "momentum_y_2": 0},
    batch_size=cfg.batch_size.Interior // 2,
    lambda_weighting={
        "continuity_2": Symbol("sdf"),
        "momentum_x_2": Symbol("sdf"),
        "momentum_y_2": Symbol("sdf"),
    },
    criteria=y > 0,  # 通过条件指定范围
)
ldc_domain.add_constraint(interior, "interior_2")

interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"continuity_1": 0, "momentum_x_1": 0, "momentum_y_1": 0},
    batch_size=cfg.batch_size.Interior // 2,
    lambda_weighting={
        "continuity_1": Symbol("sdf"),
        "momentum_x_1": Symbol("sdf"),
        "momentum_y_1": Symbol("sdf"),
    },
    criteria=y < 0,  # 通过条件指定范围
)
ldc_domain.add_constraint(interior, "interior_1")

#### 界面约束

In [14]:
# interface
interface = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec_1,
    outvar={"dirichlet_u": 0, "dirichlet_v": 0, "dirichlet_p": 0},
    batch_size=cfg.batch_size.NoSlip,
    criteria=Eq(y, 0),
)
ldc_domain.add_constraint(interface, "interface")

验证器以及其他必要组件

In [15]:
# 数据目录
file_path = "openfoam/cavity_uniformVel0.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {"Points:0": "x", "Points:1": "y", "U:0": "u", "U:1": "v", "p": "p"}
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_var["x"] += -width / 2  # center OpenFoam data
    openfoam_var["y"] += -height / 2  # center OpenFoam data
    
    # 输入数据
    openfoam_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    # 输出数据
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u", "v"]
    }
    
    # 创建validator
    openfoam_validator = PointwiseValidator(
        nodes=nodes + nodes_infer,  # 注意这个位置，推理使用率nodes_infer
        invar=openfoam_invar_numpy,
        true_outvar=openfoam_outvar_numpy,
        batch_size=1024,
        plotter=ValidatorPlotter(),
    )
    ldc_domain.add_validator(openfoam_validator)

    # 创建inferencer
    # 独立的
    samples = rec_1.sample_interior(10000)
    # add inferencer data
    grid_inference = PointwiseInferencer(
        nodes=nodes,
        invar={"x": samples["x"], "y": samples["y"]},
        output_names=["u_1", "v_1", "p_1"],
        batch_size=1024,
        plotter=InferencerPlotter(),
    )
    ldc_domain.add_inferencer(grid_inference, "inf_data_1")

    samples = rec_2.sample_interior(10000)
    # add inferencer data
    grid_inference = PointwiseInferencer(
        nodes=nodes,
        invar={"x": samples["x"], "y": samples["y"]},
        output_names=["u_2", "v_2", "p_2"],
        batch_size=1024,
        plotter=InferencerPlotter(),
    )
    ldc_domain.add_inferencer(grid_inference, "inf_data_2")
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

### 求解器以及求解

In [16]:
# 定义求解器
slv = Solver(cfg, ldc_domain)

手动加载日志系统

In [17]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [18]:
slv.solve()

2024-02-27 09:14:08,066 - modulus.sym.trainer - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-02-27 09:14:08,076 - modulus.sym.trainer - INFO - attempting to restore from: /workspace/10_2D_LDC_Domain_Decomposition/outputs
2024-02-27 09:14:08,077 - modulus.sym.trainer - WARNING - optimizer checkpoint not found
2024-02-27 09:14:08,079 - modulus.sym.trainer - WARNING - model flow_network_1.0.pth not found
2024-02-27 09:14:08,080 - modulus.sym.trainer - WARNING - model flow_network_2.0.pth not found
2024-02-27 09:14:09,806 - modulus.sym.trainer - INFO - [step:          0] record constraint batch time:  1.065e-01s
2024-02-27 09:14:16,596 - modulus.sym.trainer - INFO - [step:          0] record validators time:  6.788e+00s
2024-02-27 09:14:18,668 - modulus.sym.trainer - INFO - [step:          0] record inferencers time:  2.050e+00s
2024-02-27 09:14:18,731 - modulus.sym.trainer - INFO - [step:

2024-02-27 09:17:59,514 - modulus.sym.trainer - INFO - [step:       4600] loss:  4.099e-04, time/iteration:  3.717e+01 ms
2024-02-27 09:18:03,234 - modulus.sym.trainer - INFO - [step:       4700] loss:  1.901e-04, time/iteration:  3.718e+01 ms
2024-02-27 09:18:06,957 - modulus.sym.trainer - INFO - [step:       4800] loss:  4.157e-04, time/iteration:  3.721e+01 ms
2024-02-27 09:18:10,678 - modulus.sym.trainer - INFO - [step:       4900] loss:  4.576e-04, time/iteration:  3.720e+01 ms
2024-02-27 09:18:22,256 - modulus.sym.trainer - INFO - [step:       5000] record validators time:  6.978e+00s
2024-02-27 09:18:22,342 - modulus.sym.trainer - INFO - [step:       5000] saved checkpoint to /workspace/10_2D_LDC_Domain_Decomposition/outputs
2024-02-27 09:18:22,344 - modulus.sym.trainer - INFO - [step:       5000] loss:  3.618e-04, time/iteration:  1.166e+02 ms
2024-02-27 09:18:26,093 - modulus.sym.trainer - INFO - [step:       5100] loss:  4.247e-04, time/iteration:  3.748e+01 ms
2024-02-27 09:

2024-02-27 09:22:05,831 - modulus.sym.trainer - INFO - [step:      10000] record constraint batch time:  1.360e-01s
2024-02-27 09:22:12,728 - modulus.sym.trainer - INFO - [step:      10000] record validators time:  6.895e+00s
2024-02-27 09:22:14,828 - modulus.sym.trainer - INFO - [step:      10000] record inferencers time:  2.072e+00s
2024-02-27 09:22:14,896 - modulus.sym.trainer - INFO - [step:      10000] saved checkpoint to /workspace/10_2D_LDC_Domain_Decomposition/outputs
2024-02-27 09:22:14,898 - modulus.sym.trainer - INFO - [step:      10000] loss:  2.449e-04, time/iteration:  1.375e+02 ms
2024-02-27 09:22:14,900 - modulus.sym.trainer - INFO - [step:      10000] reached maximum training steps, finished training!


### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

如果使用了PointwiseValidator则可以直接查看验证的结果：

![u](./outputs/validators/validator_u.png)
![v](./outputs/validators/validator_v.png)

非常显著的界面过度不平滑